# silver_comercio_ext_indices.tb_referencia_ncm_unidade
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_UNIDADE_delta`
## 📌 Descrição do arquivo

Tabela de referência das **unidades estatísticas** usadas na `QT_ESTAT`.

| Coluna    | Descrição técnica | Interpretação humana                           |
| --------- | ----------------- | ---------------------------------------------- |
| `CO_UNID` | Código da unidade | Identificador usado em fatos e na dimensão NCM |
| `NO_UNID` | Nome da unidade   | Nome por extenso da unidade                    |
| `SG_UNID` | Sigla da unidade  | Abreviação usada em relatórios                 |

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:
bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/ncm_unidade/"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_unidade/"
silverTable = "silver_comercio_ext_indices.tb_referencia_ncm_unidade"


In [0]:
silverSchema = T.StructType([
    T.StructField("CO_UNID",   T.StringType(),   False),
    T.StructField("NO_UNID",   T.StringType(),   False),
    T.StructField("SG_UNID",   T.StringType(),   True ),
    T.StructField("TS_REF",    T.TimestampType(),False),
    T.StructField("NM_ORIGEM", T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**

In [0]:
def up(s): return F.upper(F.trim(F.col(s).cast(T.StringType())))

df_norm = (
    df_bronze
    .withColumn("CO_UNID", up("CO_UNID"))
    .withColumn("NO_UNID", F.col("NO_UNID").cast(T.StringType()))
    .withColumn("SG_UNID", up("SG_UNID"))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/balancacomercial/ncm_unidade"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_UNID").isNotNull())

In [0]:
df_dedup = df_valid.dropDuplicates(["CO_UNID"])

In [0]:
df_silver = df_dedup.select("CO_UNID","NO_UNID","SG_UNID","TS_REF","NM_ORIGEM")

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_indices.tb_referencia_ncm_unidade")

merge_condition = """
  t.CO_UNID = s.CO_UNID 
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "t.NO_UNID": "s.NO_UNID",
        "t.SG_UNID": "s.SG_UNID",
        "t.TS_REF": "s.TS_REF",
        "t.NM_ORIGEM": "s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute()
)